In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
import glob
import os
from astropy.io import fits
import numpy as np
import multiprocessing

import stack

np.seterr(divide='ignore', invalid='ignore')

dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/redux/v2_5_3/'
dir_dap = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/HYB10-MILESHC-MILESHC/'
dir_res = '/Users/astro/Documents/notebooks/zs/pair-m-z/stacking/'


def stacking(plateifu):
    print(plateifu)
    plate = plateifu.split('-')[0]
    ifu = plateifu.split('-')[1]
    f_map = dir_dap+plate+'/'+ifu+'/manga-'+plate + \
        '-'+ifu+'-MAPS-HYB10-MILESHC-MILESHC.fits.gz'
    f_cube = dir_drp+plate+'/stack/manga-'+plate + \
        '-'+ifu+'-LOGCUBE.fits.gz'
    if os.path.exists(f_map) and os.path.exists(f_cube):
        cube = fits.open(f_cube)
        wave = cube['WAVE'].data
        flux_t = np.transpose(cube['FLUX'].data, axes=(1, 2, 0))
        ivar_t = np.transpose(cube['IVAR'].data, axes=(1, 2, 0))

        mapf = fits.open(f_map)
        flux_map = mapf['EMLINE_GFLUX'].data
        ivar_map = mapf['EMLINE_GFLUX_IVAR'].data
        mask_map = mapf['EMLINE_GFLUX_MASK'].data
        ellcoo = mapf['SPX_ELLCOO'].data[1]
        stellar_vel = mapf['STELLAR_VEL'].data

        stack.stack(plateifu, wave, flux_t, ivar_t, flux_map, ivar_map,
                          mask_map, ellcoo, stellar_vel,dir_res)


if __name__ == '__main__':
    dir_drpall = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
    f = fits.open(dir_drpall)
    data = f[1].data
    plateifu = data.field('plateifu')
#     os.remove(dir_res+'10001-6104_1re_stack.fits')
    print('              start                  :')
    pool=multiprocessing.Pool(processes=5)
    pool.map(stacking,plateifu)
    pool.close()
    pool.join()
    print('              finished               .')



              start                  :
10001-12701
10494-9102
7495-6104
8081-6103
8146-9102
percentage of vaild points : 172 / 173
percentage of vaild points :  0.9942196531791907
(54, 54, 4563)
percentage of vaild points : 131 / 131
percentage of vaild points :  1.0
(54, 54, 4563)
plateifu 7495-6104
plateifu 8081-6103
7495-9101
8081-6104
percentage of vaild points : 194 / 348
percentage of vaild points :  0.5574712643678161
(62, 62, 4563)
plateifu 10494-9102
percentage of vaild points : 45 / 163
percentage of vaild points :  0.27607361963190186
(64, 64, 4563)
8147-12701
10496-12701
percentage of vaild points : 143 / 143
percentage of vaild points :  1.0
(74, 74, 4563)
plateifu 10001-12701
10001-12702
percentage of vaild points : 83 / 267
percentage of vaild points :  0.31086142322097376
(52, 52, 4563)
plateifu 8081-6104
8081-9101
percentage of vaild points : 212 / 213
percentage of vaild points :  0.9953051643192489
(64, 64, 4563)
plateifu 7495-9101
7495-9102
percentage of vaild point

(54, 54, 4563)
plateifu 8147-6102
8147-6103
percentage of vaild points : 317 / 317
percentage of vaild points :  1.0
(54, 54, 4563)
percentage of vaild points : 40 / 103
percentage of vaild points :  0.3883495145631068
(42, 42, 4563)
plateifu 7815-3704
plateifu 10496-6102
7815-6101
10496-6103
percentage of vaild points : 0 / 98
percentage of vaild points :  0.0
(42, 42, 4563)
8082-3704
percentage of vaild points : 115 / 325
percentage of vaild points :  0.35384615384615387
(54, 54, 4563)
plateifu 10001-6104
10001-9101
percentage of vaild points : 438 / 438
percentage of vaild points :  1.0
(54, 54, 4563)
plateifu 8147-6103
8147-6104
percentage of vaild points : 19 / 79
percentage of vaild points :  0.24050632911392406
(42, 42, 4563)
8082-6101
percentage of vaild points : 171 / 171
percentage of vaild points :  1.0
(52, 52, 4563)
plateifu 7815-6101
percentage of vaild points : 112 / 113
percentage of vaild points :  0.9911504424778761
(54, 54, 4563)
7815-6102
plateifu 10496-6103
10496-6

/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


percentage of vaild points : 156 / 279
percentage of vaild points :  0.5591397849462365
(62, 62, 4563)
plateifu 8082-9102
8083-12701
percentage of vaild points : 35 / 594
percentage of vaild points :  0.058922558922558925
(74, 74, 4563)
8148-12703


/Users/astro/.local/miniconda3/envs/public/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


...

percentage of vaild points :  0.47653429602888087
(54, 54, 4563)
plateifu 9870-6103
9488-9102
9870-6104
percentage of vaild points : 77 / 138
percentage of vaild points :  0.5579710144927537
(42, 42, 4563)
plateifu 9508-3702
9508-3703
percentage of vaild points : 327 / 1047
percentage of vaild points :  0.3123209169054441
(72, 72, 4563)
plateifu 9888-12704
9888-12705
percentage of vaild points : 207 / 264
percentage of vaild points :  0.7840909090909091
(42, 42, 4563)
plateifu 9508-3703
9508-3704
percentage of vaild points : 336 / 336
percentage of vaild points :  1.0
(54, 54, 4563)
plateifu 9870-6104
9870-9101
percentage of vaild points : 1678 / 1762
percentage of vaild points :  0.9523269012485811
(64, 64, 4563)
percentage of vaild points : 139 / 449
percentage of vaild points :  0.30957683741648107
(42, 42, 4563)
plateifu 9508-3704
9508-6101
plateifu 9488-9102
9489-12701
percentage of vaild points : 605 / 645
percentage of vaild points :  0.937984496124031
(74, 74, 4563)
percentage 

percentage of vaild points :  1.0
(42, 42, 4563)
plateifu 9509-3702
percentage of vaild points : 825 / 3212
percentage of vaild points :  0.2568493150684932
(62, 62, 4563)
9509-3703
9489-9102
percentage of vaild points : 3 / 189
percentage of vaild points :  0.015873015873015872
(54, 54, 4563)
9871-9101
percentage of vaild points : 0 / 67
percentage of vaild points :  0.0
(42, 42, 4563)
9509-3704
percentage of vaild points : 134 / 310
percentage of vaild points :  0.432258064516129
(74, 74, 4563)
plateifu 9889-12705
9889-1901
percentage of vaild points : 47 / 47
percentage of vaild points :  1.0
(34, 34, 4563)
plateifu 9889-1901
9889-1902
percentage of vaild points : 20 / 187
percentage of vaild points :  0.10695187165775401
(42, 42, 4563)
9509-6101
percentage of vaild points : 138 / 579
percentage of vaild points :  0.23834196891191708
(62, 62, 4563)
9490-12701
percentage of vaild points : 305 / 318
percentage of vaild points :  0.9591194968553459
(64, 64, 4563)
plateifu 9871-9101
987

plateifu 9510-3702
9510-3703
percentage of vaild points : 2 / 57
percentage of vaild points :  0.03508771929824561
(44, 44, 4563)
9510-3704
percentage of vaild points : 335 / 378
percentage of vaild points :  0.8862433862433863
(74, 74, 4563)
plateifu 9890-12705
percentage of vaild points : 613 / 703
percentage of vaild points :  0.871977240398293
(54, 54, 4563)
9890-1901
plateifu 9872-6104
9872-9101
percentage of vaild points : 39 / 125
percentage of vaild points :  0.312
(34, 34, 4563)
plateifu 9890-1901
9890-1902
percentage of vaild points : 35 / 419
percentage of vaild points :  0.08353221957040573
(64, 64, 4563)
9491-12701
percentage of vaild points : 117 / 192
percentage of vaild points :  0.609375
(44, 44, 4563)
plateifu 9510-3704
9510-6101
percentage of vaild points : 117 / 117
percentage of vaild points :  1.0
(34, 34, 4563)
plateifu 9890-1902
9890-3701
percentage of vaild points : 146 / 608
percentage of vaild points :  0.24013157894736842
(64, 64, 4563)
9872-9102
percentage 

(54, 54, 4563)
plateifu 9873-6104
9873-9101
percentage of vaild points : 49 / 249
percentage of vaild points :  0.19678714859437751
(74, 74, 4563)
9891-1901
percentage of vaild points : 12 / 93
percentage of vaild points :  0.12903225806451613
(42, 42, 4563)
9511-3704
percentage of vaild points : 216 / 216
percentage of vaild points :  1.0
(64, 64, 4563)
plateifu 9491-9102
9492-12701
percentage of vaild points : 0 / 41
percentage of vaild points :  0.0
(34, 34, 4563)
9891-1902
percentage of vaild points : 52 / 67
percentage of vaild points :  0.7761194029850746
(42, 42, 4563)
plateifu 9511-3704
9511-6101
percentage of vaild points : 81 / 167
percentage of vaild points :  0.48502994011976047
(34, 34, 4563)
plateifu 9891-1902
9891-3701
percentage of vaild points : 1 / 68
percentage of vaild points :  0.014705882352941176
(64, 64, 4563)
9873-9102
percentage of vaild points : 23 / 623
percentage of vaild points :  0.03691813804173355
(44, 44, 4563)
9891-3702
percentage of vaild points : 2 

(72, 72, 4563)
plateifu 9892-12705
9892-1901
percentage of vaild points : 0 / 164
percentage of vaild points :  0.0
(42, 42, 4563)
9512-3704
percentage of vaild points : 0 / 46
percentage of vaild points :  0.0
(32, 32, 4563)
9892-1902
percentage of vaild points : 452 / 491
percentage of vaild points :  0.9205702647657841
(64, 64, 4563)
plateifu 9492-9102
9493-12701
percentage of vaild points : 16 / 202
percentage of vaild points :  0.07920792079207921
(42, 42, 4563)
9512-6101
percentage of vaild points : 23 / 30
percentage of vaild points :  0.7666666666666667
(32, 32, 4563)
plateifu 9892-1902
9892-3701
percentage of vaild points : 38 / 439
percentage of vaild points :  0.08656036446469248
(64, 64, 4563)
9875-12701
percentage of vaild points : 125 / 245
percentage of vaild points :  0.5102040816326531
(42, 42, 4563)
plateifu 9892-3701
9892-3702
percentage of vaild points : 43 / 145
percentage of vaild points :  0.296551724137931
(52, 52, 4563)
9512-6102
percentage of vaild points : 21

In [2]:
a=np.array([21,4,5,6,7])
mask=(a>4)
a[mask]


array([21,  5,  6,  7])

In [25]:
a=np.array([21,4,5,6,7])
mask=(a>4)
c=np.ma.array(a,mask=mask)
c*(~mask)
c[0]

masked